In [ ]:
@everywhere using SIS,IM,PayloadGraph,PyPlot, Epidemics
import LightGraphs

@everywhere begin

k = 10
#y_n = 0.1
c_r = 0.18
N = 1000
n_n = 400#y_n*N
beta = 4.0/(c_r*n_n)
alpha = (N*beta)/n_n
println(N, ' ' ,alpha, ' ',beta)

im = InfectionModel(x -> 1 + alpha*x , x -> 1 + beta);
imk = InfectionModel(x -> 1 + beta + get_s_eff(x,alpha,beta,k) , x -> 1 + beta);

y_n, y_minus, y_plus, y_p,critical_determinant = get_parameters(N,alpha,beta)
num_trials = 1;regular=false;
end

In [ ]:
if regular
    g = Epidemics.guarantee_connected( () -> LightGraphs.random_regular_graph(N,k))
else
    g = Epidemics.guarantee_connected( () -> LightGraphs.erdos_renyi(N,1.0*k/(N-1)))
end
p = create_graph_from_value(g,INFECTED)
infecteds = []
set_payload(p,1,SUSCEPTIBLE)
frac = SIS.get_fraction_of_type(p,INFECTED)
push!(infecteds,N*frac)
new_types = convert(SharedArray,fill(SUSCEPTIBLE,N))
count = 0
while p.payload[1] == 0 && count < 10
    println(new_types,p)
    update_graph(p,im,new_types)
    count += 1
end

In [ ]:
Profile.clear()
srand(1)
@time sizes,num_fixed,_,runs = 
    run_epidemics(num_trials,im, (x)
    -> run_epidemic_graph(N,k,x,regular,1.0))



In [ ]:
using ProfileView
num_trials = 1
procs()

In [ ]:
1+1